In [ ]:
# Author: William Chuang
# Last modified: Sep 26, 2023
# This code is modified and built on the code written by Dr. Phillip Lippe.
#
## Standard libraries
import os
import numpy as np
import random
import math
import json
from functools import partial


## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim


# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "./data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "./saved_models/tutorial6"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


Device: cpu


In [ ]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    #print(d_k)
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(0.005*d_k) #0.005 #10 #3 #1.414 #(0.00001*d_k) #(d_k)**(1/100) #math.sqrt(d_k*2)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [ ]:
class MultiheadAttention(nn.Module):

    def __init__(self, input_dim, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Stack all weight matrices 1...h together for efficiency
        # Note that in many implementations you see "bias=False" which is optional
        self.qkv_proj = nn.Linear(input_dim, 3*embed_dim)
        self.o_proj = nn.Linear(embed_dim, embed_dim)

        self._reset_parameters()

    def _reset_parameters(self):
        # Original Transformer initialization, see PyTorch documentation
        nn.init.xavier_uniform_(self.qkv_proj.weight)
        self.qkv_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.o_proj.weight)
        self.o_proj.bias.data.fill_(0)

    def forward(self, x, mask=None, return_attention=False):
        batch_size, seq_length, _ = x.size()
        if mask is not None:
            mask = expand_mask(mask)
        qkv = self.qkv_proj(x)

        # Separate Q, K, V from linear output
        qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)

        # Determine value outputs
        values, attention = scaled_dot_product(q, k, v, mask=mask)
        values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        values = values.reshape(batch_size, seq_length, self.embed_dim)
        o = self.o_proj(values)

        if return_attention:
            return o, attention
        else:
            return o

In [ ]:
class EncoderBlock(nn.Module):

    def __init__(self, input_dim, num_heads, dim_feedforward, dropout=0.0):
        """
        Inputs:
            input_dim - Dimensionality of the input
            num_heads - Number of heads to use in the attention block
            dim_feedforward - Dimensionality of the hidden layer in the MLP
            dropout - Dropout probability to use in the dropout layers
        """
        super().__init__()

        # Attention layer
        self.self_attn = MultiheadAttention(input_dim, input_dim, num_heads)

        # Two-layer MLP
        self.linear_net = nn.Sequential(
            nn.Linear(input_dim, dim_feedforward),
            nn.Dropout(dropout),
            nn.ReLU(inplace=True),
            nn.Linear(dim_feedforward, input_dim)
        )

        # Layers to apply in between the main layers
        self.norm1 = nn.LayerNorm(input_dim)
        self.norm2 = nn.LayerNorm(input_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        # Attention part
        attn_out = self.self_attn(x, mask=mask)
        x = x + self.dropout(attn_out)
        x = self.norm1(x)

        # MLP part
        linear_out = self.linear_net(x)
        x = x + self.dropout(linear_out)
        x = self.norm2(x)

        return x

In [ ]:
class TransformerEncoder(nn.Module):

    def __init__(self, num_layers, **block_args):
        super().__init__()
        self.layers = nn.ModuleList([EncoderBlock(**block_args) for _ in range(num_layers)])

    def forward(self, x, mask=None):
        for l in self.layers:
            x = l(x, mask=mask)
        return x

    def get_attention_maps(self, x, mask=None):
        attention_maps = []
        for l in self.layers:
            _, attn_map = l.self_attn(x, mask=mask, return_attention=True)
            attention_maps.append(attn_map)
            x = l(x)
        return attention_maps

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        # register_buffer => Tensor which is not a parameter, but should be part of the modules state.
        # Used for tensors that need to be on the same device as the module.
        # persistent=False tells PyTorch to not add the buffer to the state dict (e.g. when we save the model)
        self.register_buffer('pe', pe, persistent=False)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

In [ ]:
class TransformerPredictor(pl.LightningModule):

    def __init__(self, input_dim, model_dim, num_classes, num_heads, num_layers, lr, warmup, max_iters, dropout=0.0, input_dropout=0.0):
        """
        Inputs:
            input_dim - Hidden dimensionality of the input
            model_dim - Hidden dimensionality to use inside the Transformer
            num_classes - Number of classes to predict per sequence element
            num_heads - Number of heads to use in the Multi-Head Attention blocks
            num_layers - Number of encoder blocks to use.
            lr - Learning rate in the optimizer
            warmup - Number of warmup steps. Usually between 50 and 500
            max_iters - Number of maximum iterations the model is trained for. This is needed for the CosineWarmup scheduler
            dropout - Dropout to apply inside the model
            input_dropout - Dropout to apply on the input features
        """
        super().__init__()
        self.save_hyperparameters()
        self._create_model()

    def _create_model(self):
        # Input dim -> Model dim
        self.input_net = nn.Sequential(
            nn.Dropout(self.hparams.input_dropout),
            nn.Linear(self.hparams.input_dim, self.hparams.model_dim)
        )
        # Positional encoding for sequences
        self.positional_encoding = PositionalEncoding(d_model=self.hparams.model_dim)
        # Transformer
        self.transformer = TransformerEncoder(num_layers=self.hparams.num_layers,
                                              input_dim=self.hparams.model_dim,
                                              dim_feedforward=2*self.hparams.model_dim,
                                              num_heads=self.hparams.num_heads,
                                              dropout=self.hparams.dropout)
        # Output classifier per sequence lement
        self.output_net = nn.Sequential(
            nn.Linear(self.hparams.model_dim, self.hparams.model_dim),
            nn.LayerNorm(self.hparams.model_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(self.hparams.dropout),
            nn.Linear(self.hparams.model_dim, self.hparams.num_classes)
        )

    def forward(self, x, mask=None, add_positional_encoding=True):
        """
        Inputs:
            x - Input features of shape [Batch, SeqLen, input_dim]
            mask - Mask to apply on the attention outputs (optional)
            add_positional_encoding - If True, we add the positional encoding to the input.
                                      Might not be desired for some tasks.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        x = self.transformer(x, mask=mask)
        x = self.output_net(x)
        return x

    @torch.no_grad()
    def get_attention_maps(self, x, mask=None, add_positional_encoding=True):
        """
        Function for extracting the attention matrices of the whole Transformer for a single batch.
        Input arguments same as the forward pass.
        """
        x = self.input_net(x)
        if add_positional_encoding:
            x = self.positional_encoding(x)
        attention_maps = self.transformer.get_attention_maps(x, mask=mask)
        return attention_maps

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.hparams.lr)

        # Apply lr scheduler per step
        lr_scheduler = CosineWarmupScheduler(optimizer,
                                             warmup=self.hparams.warmup,
                                             max_iters=self.hparams.max_iters)
        return [optimizer], [{'scheduler': lr_scheduler, 'interval': 'step'}]

    def training_step(self, batch, batch_idx):
        raise NotImplementedError

    def validation_step(self, batch, batch_idx):
        raise NotImplementedError

    def test_step(self, batch, batch_idx):
        raise NotImplementedError

In [ ]:
class ReverseDataset(data.Dataset):

    def __init__(self, num_categories, seq_len, size):
        super().__init__()
        self.num_categories = num_categories

        self.seq_len = seq_len
        self.size = size

        self.data = torch.randint(self.num_categories, size=(self.size, self.seq_len))
        # self.data = torch.abs(torch.normal(0, 1, size=(self.size, self.seq_len)).long())
        # torch.randint(low=0, high, size, \*, generator=None, out=None, dtype=None,
        # layout=torch.strided, device=None, requires_grad=False) → Tensor
        print(self.num_categories)
        print(self.seq_len)
        print(self.size)
        print(self.data)

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        inp_data = self.data[idx]
        labels = torch.flip(inp_data, dims=(0,))
        return inp_data, labels
#''' Examples of torch.randint
#>>> torch.randint(3, 5, (3,))
#tensor([4, 3, 4])


#>>> torch.randint(10, (2, 2))
#tensor([[0, 2],
#        [5, 5]])


#>>> torch.randint(3, 10, (2, 2))
#tensor([[4, 5],
#        [6, 7]])'''

#'''>>> torch.normal(mean=0.5, std=torch.arange(1., 6.))
#tensor([-1.2793, -1.0732, -2.0687,  5.1177, -1.2303])'''

In [ ]:
class ReversePredictor(TransformerPredictor):

    def _calculate_loss(self, batch, mode="train"):
        # Fetch data and transform categories to one-hot vectors
        inp_data, labels = batch
        inp_data = F.one_hot(inp_data, num_classes=self.hparams.num_classes).float()

        # Perform prediction and calculate loss and accuracy
        preds = self.forward(inp_data, add_positional_encoding=True)
        loss = F.cross_entropy(preds.view(-1,preds.size(-1)), labels.view(-1))
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        # Logging
        self.log(f"{mode}_loss", loss)
        self.log(f"{mode}_acc", acc)
        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, _ = self._calculate_loss(batch, mode="train")
        return loss

    def validation_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="val")

    def test_step(self, batch, batch_idx):
        _ = self._calculate_loss(batch, mode="test")

In [ ]:
def train_reverse(**kwargs):
    # Create a PyTorch Lightning trainer with the generation callback
    root_dir = os.path.join(CHECKPOINT_PATH, "ReverseTask")
    os.makedirs(root_dir, exist_ok=True)
    trainer = pl.Trainer(default_root_dir=root_dir,
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=10,
                         gradient_clip_val=5)
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, "ReverseTask.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = ReversePredictor.load_from_checkpoint(pretrained_filename)
    else:
        print("Found pretrained model does not exist, generating...")
        model = ReversePredictor(max_iters=trainer.max_epochs*len(train_loader), **kwargs)
        trainer.fit(model, train_loader, val_loader)

    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"test_acc": test_result[0]["test_acc"], "val_acc": val_result[0]["test_acc"]}

    model = model.to(device)
    return model, result

In [ ]:
class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor

In [ ]:
dataset = partial(ReverseDataset, 10, 20)
train_loader = data.DataLoader(dataset(8000), batch_size=128, shuffle=True, drop_last=True, pin_memory=True)
val_loader   = data.DataLoader(dataset(1000), batch_size=128)
test_loader  = data.DataLoader(dataset(20000), batch_size=128)

10
20
8000
tensor([[2, 7, 6,  ..., 5, 7, 6],
        [9, 6, 3,  ..., 6, 2, 0],
        [6, 2, 7,  ..., 4, 8, 8],
        ...,
        [0, 6, 4,  ..., 8, 2, 4],
        [4, 4, 6,  ..., 8, 2, 4],
        [5, 2, 3,  ..., 3, 7, 5]])
10
20
1000
tensor([[3, 1, 2,  ..., 2, 9, 6],
        [7, 2, 3,  ..., 2, 5, 8],
        [5, 9, 1,  ..., 9, 3, 1],
        ...,
        [7, 4, 1,  ..., 1, 0, 5],
        [6, 0, 5,  ..., 9, 3, 0],
        [4, 4, 3,  ..., 3, 0, 6]])
10
20
20000
tensor([[0, 4, 3,  ..., 1, 0, 2],
        [1, 1, 7,  ..., 2, 7, 3],
        [9, 7, 2,  ..., 6, 8, 4],
        ...,
        [6, 7, 1,  ..., 1, 9, 6],
        [0, 1, 5,  ..., 4, 3, 9],
        [6, 1, 5,  ..., 3, 2, 4]])


In [ ]:
inp_data, labels = train_loader.dataset[0]
print("Input data:", inp_data)
print("Labels:    ", labels)

Input data: tensor([2, 7, 6, 4, 6, 5, 0, 4, 0, 3, 8, 4, 0, 4, 1, 2, 5, 5, 7, 6])
Labels:     tensor([6, 7, 5, 5, 2, 1, 4, 0, 4, 8, 3, 0, 4, 0, 5, 6, 4, 6, 7, 2])


In [ ]:
reverse_model, reverse_result = train_reverse(input_dim=train_loader.dataset.num_categories,
                                              model_dim=20,
                                              num_heads=1,
                                              num_classes=train_loader.dataset.num_categories,
                                              num_layers=1,
                                              dropout=0.0,
                                              lr=5e-4,
                                              warmup=50)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Found pretrained model does not exist, generating...


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Sequential         | 220   
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 3.4 K 
3 | output_net          | Sequential         | 670   
-----------------------------------------------------------
4.3 K     Trainable params
0         Non-trainable params
4.3 K     Total params
0.017     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

In [ ]:
# @title
print(f"Val accuracy:  {(100.0 * reverse_result['val_acc']):4.2f}%")
print(f"Test accuracy: {(100.0 * reverse_result['test_acc']):4.2f}%")

Val accuracy:  95.09%
Test accuracy: 95.24%


In [ ]:
# multiplied by d_k^0.5
for i in range(10):
  reverse_model, reverse_result = train_reverse(input_dim=train_loader.dataset.num_categories,
                                              model_dim=20,
                                              num_heads=1,
                                              num_classes=train_loader.dataset.num_categories,
                                              num_layers=1,
                                              dropout=0.0,
                                              lr=5e-4,
                                              warmup=50)
  print(f"Val accuracy:  {(100.0 * reverse_result['val_acc']):4.2f}%")
  print(f"Test accuracy: {(100.0 * reverse_result['test_acc']):4.2f}%")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Sequential         | 220   
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 3.4 K 
3 | output_net          | Sequential         | 670   
-----------------------------------------------------------
4.3 K     Trainable params
0         Non-trainable params
4.3 K     Total params
0.017     Total estimated model params size (MB)


Found pretrained model does not exist, generating...


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

Val accuracy:  26.06%
Test accuracy: 25.80%


In [ ]:
# multiplied by (0.005*d_k)^0.5
for i in range(10):
  reverse_model, reverse_result = train_reverse(input_dim=train_loader.dataset.num_categories,
                                              model_dim=20,
                                              num_heads=1,
                                              num_classes=train_loader.dataset.num_categories,
                                              num_layers=1,
                                              dropout=0.0,
                                              lr=5e-4,
                                              warmup=50)
  print(f"Val accuracy:  {(100.0 * reverse_result['val_acc']):4.2f}%")
  print(f"Test accuracy: {(100.0 * reverse_result['test_acc']):4.2f}%")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Sequential         | 220   
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 3.4 K 
3 | output_net          | Sequential         | 670   
-----------------------------------------------------------
4.3 K     Trainable params
0         Non-trainable params
4.3 K     Total params
0.017     Total estimated model params size (MB)


Found pretrained model does not exist, generating...


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

Val accuracy:  92.84%
Test accuracy: 93.08%


In [ ]:
reverse_result

{'test_acc': 1.0, 'val_acc': 1.0}

In [ ]:
PATH = "/content/saved_models/tutorial6/ReverseTask/lightning_logs/version_122/checkpoints/epoch=9-step=620.ckpt"
eooch_1_model = torch.load(PATH)

In [ ]:
print(eooch_1_model)

{'epoch': 9, 'global_step': 620, 'pytorch-lightning_version': '2.0.9', 'state_dict': OrderedDict([('input_net.1.weight', tensor([[-6.1192e-02, -5.7457e-02, -2.0628e-01, -2.8326e-04,  1.9193e-01,
          1.1146e-01, -1.1664e-01, -3.8459e-02,  5.9248e-02,  9.3121e-02],
        [-5.6065e-02, -1.2265e-01, -1.4355e-01,  1.2432e-01,  9.8715e-02,
         -6.4473e-02, -2.3869e-01,  1.0493e-01,  1.9670e-01, -9.5173e-02],
        [-1.1283e-01, -2.0046e-01,  1.5611e-01,  4.2801e-01, -6.1218e-02,
          1.7485e-01, -3.2932e-01, -3.4412e-02, -7.2208e-02, -1.1475e-01],
        [ 1.0397e-01,  3.6135e-01, -2.6397e-01, -3.0885e-01, -2.9126e-01,
          1.7955e-02, -1.2191e-01,  5.4734e-02, -1.3674e-01,  2.7720e-01],
        [ 1.8761e-01,  2.5501e-01, -1.3681e-01, -8.5685e-02,  5.1108e-02,
          5.3949e-02,  1.7983e-01, -9.6947e-02, -1.9027e-01,  1.9366e-01],
        [ 2.2769e-01,  3.5281e-02, -2.1033e-01, -1.6461e-01, -1.4656e-01,
          6.5002e-02,  2.9095e-01, -7.0525e-02, -2.4532e-01,

In [ ]:
print(reverse_model)

ReversePredictor(
  (input_net): Sequential(
    (0): Dropout(p=0.0, inplace=False)
    (1): Linear(in_features=10, out_features=20, bias=True)
  )
  (positional_encoding): PositionalEncoding()
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): EncoderBlock(
        (self_attn): MultiheadAttention(
          (qkv_proj): Linear(in_features=20, out_features=60, bias=True)
          (o_proj): Linear(in_features=20, out_features=20, bias=True)
        )
        (linear_net): Sequential(
          (0): Linear(in_features=20, out_features=40, bias=True)
          (1): Dropout(p=0.0, inplace=False)
          (2): ReLU(inplace=True)
          (3): Linear(in_features=40, out_features=20, bias=True)
        )
        (norm1): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((20,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (output_net): Sequential(
    (0): Linear(in_features=20, o

In [ ]:
PATH = "./new.pth"
torch.save(reverse_model.state_dict(), PATH)

In [ ]:
PATH = "./new.pth"
w=torch.load(PATH)

In [ ]:
type(w)

collections.OrderedDict

In [ ]:
import pprint
pprint.pprint(w, sort_dicts=False)

OrderedDict([('input_net.1.weight',
              tensor([[-1.3998e-01, -6.5229e-02, -1.0811e-01,  1.3060e-01,  1.2438e-01,
         -5.2942e-03,  1.6225e-01, -5.2731e-03, -1.6932e-02,  1.2357e-01],
        [-5.4484e-03, -4.0976e-02, -1.7412e-01, -3.9024e-02, -2.9362e-01,
          6.0723e-02, -1.6704e-01, -1.7556e-01, -1.0406e-01, -8.0133e-02],
        [ 2.0050e-01,  1.3809e-02, -1.3877e-01, -1.1548e-01, -9.9193e-02,
          3.4606e-01, -5.3908e-02, -7.2678e-02, -1.4170e-01, -1.3008e-01],
        [-7.1332e-02, -7.5434e-02,  6.2920e-02,  9.0777e-02,  8.4999e-02,
         -1.5283e-01,  2.0562e-01,  1.3133e-01,  9.7419e-03,  3.4302e-02],
        [ 2.3101e-01,  2.8362e-01,  2.8380e-01,  2.6599e-01,  1.2737e-01,
          5.3119e-02,  7.5725e-02, -1.8625e-01, -1.6634e-03, -3.3357e-01],
        [ 1.1607e-01,  2.5535e-01, -8.1673e-02,  8.5079e-02,  3.6482e-01,
          1.0600e-01,  1.3852e-01,  2.7283e-01,  2.7659e-01, -1.8831e-01],
        [ 3.0286e-01, -8.8423e-02,  1.1058e-01,  6.6554e

In [ ]:
#for odict in w:
#    print(odict)
for odict in w:
    print(str(odict)+": "+str(len(w[str(odict)])))
    #print(str( (w[str(odict)][5])))


input_net.1.weight: 20
input_net.1.bias: 20
transformer.layers.0.self_attn.qkv_proj.weight: 60
transformer.layers.0.self_attn.qkv_proj.bias: 60
transformer.layers.0.self_attn.o_proj.weight: 20
transformer.layers.0.self_attn.o_proj.bias: 20
transformer.layers.0.linear_net.0.weight: 40
transformer.layers.0.linear_net.0.bias: 40
transformer.layers.0.linear_net.3.weight: 20
transformer.layers.0.linear_net.3.bias: 20
transformer.layers.0.norm1.weight: 20
transformer.layers.0.norm1.bias: 20
transformer.layers.0.norm2.weight: 20
transformer.layers.0.norm2.bias: 20
output_net.0.weight: 20
output_net.0.bias: 20
output_net.1.weight: 20
output_net.1.bias: 20
output_net.4.weight: 10
output_net.4.bias: 10


In [ ]:
print(w["input_net.1.weight"])

tensor([[-1.3998e-01, -6.5229e-02, -1.0811e-01,  1.3060e-01,  1.2438e-01,
         -5.2942e-03,  1.6225e-01, -5.2731e-03, -1.6932e-02,  1.2357e-01],
        [-5.4484e-03, -4.0976e-02, -1.7412e-01, -3.9024e-02, -2.9362e-01,
          6.0723e-02, -1.6704e-01, -1.7556e-01, -1.0406e-01, -8.0133e-02],
        [ 2.0050e-01,  1.3809e-02, -1.3877e-01, -1.1548e-01, -9.9193e-02,
          3.4606e-01, -5.3908e-02, -7.2678e-02, -1.4170e-01, -1.3008e-01],
        [-7.1332e-02, -7.5434e-02,  6.2920e-02,  9.0777e-02,  8.4999e-02,
         -1.5283e-01,  2.0562e-01,  1.3133e-01,  9.7419e-03,  3.4302e-02],
        [ 2.3101e-01,  2.8362e-01,  2.8380e-01,  2.6599e-01,  1.2737e-01,
          5.3119e-02,  7.5725e-02, -1.8625e-01, -1.6634e-03, -3.3357e-01],
        [ 1.1607e-01,  2.5535e-01, -8.1673e-02,  8.5079e-02,  3.6482e-01,
          1.0600e-01,  1.3852e-01,  2.7283e-01,  2.7659e-01, -1.8831e-01],
        [ 3.0286e-01, -8.8423e-02,  1.1058e-01,  6.6554e-02,  3.9885e-01,
         -3.6541e-02, -1.0117e-0

In [ ]:
print(w["input_net.1.bias"])

tensor([-0.1295,  0.0978, -0.1804, -0.0376,  0.1900, -0.1376, -0.2911,  0.0264,
         0.3373, -0.2690,  0.0618, -0.2595, -0.1939, -0.1999,  0.1451, -0.1385,
        -0.1783,  0.0685,  0.2269, -0.2667], device='cuda:0')


In [ ]:
print(w["transformer.layers.0.self_attn.qkv_proj.weight"])

tensor([[ 0.3366, -0.1189, -0.2746,  ...,  0.1380, -0.0323, -0.1484],
        [-0.2731, -0.2253,  0.3358,  ..., -0.1017,  0.0900,  0.0592],
        [ 0.0632, -0.2878,  0.2216,  ..., -0.2646, -0.0712,  0.2453],
        ...,
        [-0.1648,  0.1777,  0.0198,  ..., -0.2178,  0.1017,  0.1118],
        [ 0.1091,  0.0248,  0.2418,  ..., -0.0932, -0.0405, -0.0572],
        [-0.2824, -0.1440, -0.0951,  ..., -0.2419,  0.0720,  0.0053]],
       device='cuda:0')


In [ ]:
print(len(w["transformer.layers.0.self_attn.qkv_proj.weight"]))

60


In [ ]:
print(w["transformer.layers.0.self_attn.qkv_proj.weight"][59])

tensor([-0.2824, -0.1440, -0.0951, -0.0759,  0.3464, -0.3512, -0.1667, -0.1982,
        -0.2034,  0.2408, -0.3670, -0.1522, -0.1793, -0.2372,  0.0765,  0.2060,
        -0.3583, -0.2419,  0.0720,  0.0053], device='cuda:0')


In [ ]:
for odict in w:
    print(len(w[str(odict)]))

20
20
60
60
20
20
40
40
20
20
20
20
20
20
20
20
20
20
10
10


In [ ]:
for odict in w:
    print(odict)
    print(w[str(odict)])
    #print(str( (w[str(odict)][5])))
    print("-------------------------------------------------------------------------------")

input_net.1.weight
tensor([[-1.3998e-01, -6.5229e-02, -1.0811e-01,  1.3060e-01,  1.2438e-01,
         -5.2942e-03,  1.6225e-01, -5.2731e-03, -1.6932e-02,  1.2357e-01],
        [-5.4484e-03, -4.0976e-02, -1.7412e-01, -3.9024e-02, -2.9362e-01,
          6.0723e-02, -1.6704e-01, -1.7556e-01, -1.0406e-01, -8.0133e-02],
        [ 2.0050e-01,  1.3809e-02, -1.3877e-01, -1.1548e-01, -9.9193e-02,
          3.4606e-01, -5.3908e-02, -7.2678e-02, -1.4170e-01, -1.3008e-01],
        [-7.1332e-02, -7.5434e-02,  6.2920e-02,  9.0777e-02,  8.4999e-02,
         -1.5283e-01,  2.0562e-01,  1.3133e-01,  9.7419e-03,  3.4302e-02],
        [ 2.3101e-01,  2.8362e-01,  2.8380e-01,  2.6599e-01,  1.2737e-01,
          5.3119e-02,  7.5725e-02, -1.8625e-01, -1.6634e-03, -3.3357e-01],
        [ 1.1607e-01,  2.5535e-01, -8.1673e-02,  8.5079e-02,  3.6482e-01,
          1.0600e-01,  1.3852e-01,  2.7283e-01,  2.7659e-01, -1.8831e-01],
        [ 3.0286e-01, -8.8423e-02,  1.1058e-01,  6.6554e-02,  3.9885e-01,
         -3.6

In [ ]:
train_loader.dataset[0]

(tensor([7, 3, 5, 3, 4, 3, 5, 8, 0, 0, 1, 9, 5, 1, 4, 5, 9, 0, 1, 5]),
 tensor([5, 1, 0, 9, 5, 4, 1, 5, 9, 1, 0, 0, 8, 5, 3, 4, 3, 5, 3, 7]))

In [ ]:
for i in train_loader.dataset[0:3]:
  print(i)

tensor([[7, 3, 5, 3, 4, 3, 5, 8, 0, 0, 1, 9, 5, 1, 4, 5, 9, 0, 1, 5],
        [1, 1, 4, 3, 5, 0, 1, 0, 1, 7, 7, 6, 0, 6, 1, 6, 0, 5, 7, 1],
        [6, 0, 5, 1, 6, 5, 7, 8, 9, 6, 2, 6, 7, 6, 8, 5, 0, 7, 7, 9]])
tensor([[6, 0, 5, 1, 6, 5, 7, 8, 9, 6, 2, 6, 7, 6, 8, 5, 0, 7, 7, 9],
        [1, 1, 4, 3, 5, 0, 1, 0, 1, 7, 7, 6, 0, 6, 1, 6, 0, 5, 7, 1],
        [7, 3, 5, 3, 4, 3, 5, 8, 0, 0, 1, 9, 5, 1, 4, 5, 9, 0, 1, 5]])
